In [1]:
import torch
from torch_geometric.datasets import TUDataset

from GNN import GCN_Mutag

torch.manual_seed(12345)

In [2]:
dataset = TUDataset(root='data/TUDataset', name='MUTAG')

print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')


Dataset: MUTAG(188):
Number of graphs: 188
Number of features: 7
Number of classes: 2

Data(edge_index=[2, 38], x=[17, 7], edge_attr=[38, 4], y=[1])
Number of nodes: 17
Number of edges: 38
Average node degree: 2.24
Has isolated nodes: False
Has self-loops: False
Is undirected: True


In [3]:
dataset = dataset.shuffle()

train_dataset = dataset[:150]
test_dataset = dataset[150:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

Number of training graphs: 150
Number of test graphs: 38


In [4]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 2636], x=[1188, 7], edge_attr=[2636, 4], y=[64], batch=[1188], ptr=[65])

Step 2:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 2506], x=[1139, 7], edge_attr=[2506, 4], y=[64], batch=[1139], ptr=[65])

Step 3:
Number of graphs in the current batch: 22
DataBatch(edge_index=[2, 852], x=[387, 7], edge_attr=[852, 4], y=[22], batch=[387], ptr=[23])



In [5]:
model = GCN_Mutag(
    in_features=dataset.num_node_features,
    h_features=128,
    n_classes=2
)
print(model)

GCN_Mutag(
  (conv1): GraphConvolution (7 -> 128)
  (conv2): GraphConvolution (128 -> 128)
  (conv3): GraphConvolution (128 -> 128)
  (dense1): Linear(in_features=128, out_features=16, bias=True)
  (dense2): Linear(in_features=16, out_features=8, bias=True)
  (dense3): Linear(in_features=8, out_features=1, bias=True)
)


In [33]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCELoss()

def train():
    model.train()
    for data in train_loader:  # Iterate in batches over the training dataset.
        out = model(data.x, data.edge_index, data.batch).squeeze()  # Perform a single forward pass.
        loss = criterion(out, data.y.float()) # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.

def test(loader):
    model.eval()
    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data.x, data.edge_index, data.batch).squeeze()  
        pred = out.round()
        # pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.

In [36]:
best_test_acc = 0.0
for epoch in range(1, 100):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')
    if test_acc >= best_test_acc:
        best_test_acc = test_acc
        best_model_params = model.state_dict()
        print("Checkpoint saved!")

Epoch: 001, Train Acc: 0.7600, Test Acc: 0.7895
Checkpoint saved!
Epoch: 002, Train Acc: 0.7467, Test Acc: 0.7368
Epoch: 003, Train Acc: 0.7400, Test Acc: 0.8158
Checkpoint saved!
Epoch: 004, Train Acc: 0.7600, Test Acc: 0.7105
Epoch: 005, Train Acc: 0.7533, Test Acc: 0.8158
Checkpoint saved!
Epoch: 006, Train Acc: 0.7600, Test Acc: 0.6579
Epoch: 007, Train Acc: 0.7467, Test Acc: 0.7895
Epoch: 008, Train Acc: 0.7467, Test Acc: 0.7895
Epoch: 009, Train Acc: 0.7600, Test Acc: 0.7105
Epoch: 010, Train Acc: 0.7600, Test Acc: 0.8158
Checkpoint saved!
Epoch: 011, Train Acc: 0.7533, Test Acc: 0.8158
Checkpoint saved!
Epoch: 012, Train Acc: 0.7733, Test Acc: 0.8158
Checkpoint saved!
Epoch: 013, Train Acc: 0.7533, Test Acc: 0.7895
Epoch: 014, Train Acc: 0.7600, Test Acc: 0.7895
Epoch: 015, Train Acc: 0.7667, Test Acc: 0.8158
Checkpoint saved!
Epoch: 016, Train Acc: 0.7667, Test Acc: 0.8158
Checkpoint saved!
Epoch: 017, Train Acc: 0.7800, Test Acc: 0.7632
Epoch: 018, Train Acc: 0.7800, Test Acc:

## Rough

In [ ]:
dataset[0]

In [22]:
m = torch.nn.Sigmoid()
loss = torch.nn.BCELoss()
input = torch.Tensor([0.0, 1.0, 0.0])
target = torch.Tensor([0., 1., 0.])
output = loss(input, target)

In [23]:
output

tensor(0.)